In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("/content/drive/MyDrive/SWIG/보성군_AWS_및_기상_변수선정_및_전처리본_Eng.csv")

columns_to_convert = data.columns[data.columns != 'ds']
data[columns_to_convert] = data[columns_to_convert].astype(float)

In [2]:
print(data['y'].describe())

count    3768.000000
mean       22.829442
std        12.197139
min         0.535856
25%        12.841180
50%        20.805655
75%        32.172835
max        62.995488
Name: y, dtype: float64


In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from prophet import Prophet

data['y'] = np.log(data['y'])

# 훈련 데이터셋과 테스트 데이터셋으로 나누기
train_df, test_df = train_test_split(data, test_size=0.2, shuffle=False)

# 하이퍼파라미터 그리드를 정의합니다.
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.01, 0.1, 1.0, 10.0]
}

# 최적의 하이퍼파라미터와 그 때의 RMSE를 저장할 변수를 초기화합니다.
best_params = None
best_rmse = np.inf

# 모든 하이퍼파라미터 조합에 대해 탐색을 수행합니다.
for changepoint_prior_scale in param_grid['changepoint_prior_scale']:
    for seasonality_prior_scale in param_grid['seasonality_prior_scale']:
        for holidays_prior_scale in param_grid['holidays_prior_scale']:
            # 모델을 생성하고 학습합니다.
            model = Prophet(changepoint_prior_scale=changepoint_prior_scale,
                            seasonality_prior_scale=seasonality_prior_scale,
                            holidays_prior_scale=holidays_prior_scale)
            model.add_regressor('AvgT')
            model.add_regressor('MaxWdSd')
            model.add_regressor('AvgHu')
            model.add_regressor('Sum_DL')
            model.add_regressor('50CM_SM')
            model.add_regressor('Avg_GT')
            model.add_regressor('Avg_IM')
            model.add_regressor('Rain')
            model.fit(train_df)

            # 예측을 수행합니다.
            future = model.make_future_dataframe(periods=len(test_df), freq='D', include_history=True)
            for col in columns_to_convert:
                if col != 'y':
                    future[col] = data[col]
            forecast = model.predict(future)

            # RMSE를 계산합니다.
            rmse = np.sqrt(mean_squared_error(np.exp(test_df['y']), np.exp(forecast.iloc[-len(test_df):]['yhat'])))

            # 현재의 하이퍼파라미터가 더 좋은 성능을 보이면, 최적의 하이퍼파라미터를 업데이트합니다.
            if rmse < best_rmse:
                best_params = {'changepoint_prior_scale': changepoint_prior_scale,
                               'seasonality_prior_scale': seasonality_prior_scale,
                               'holidays_prior_scale': holidays_prior_scale}
                best_rmse = rmse

# 최적의 하이퍼파라미터와 그 때의 RMSE를 출력합니다.
print("최적의 하이퍼파라미터:", best_params)
print("최적의 RMSE:", best_rmse)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpiv_prys6/rcl8d6xb.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpiv_prys6/5ejzaf4r.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=75175', 'data', 'file=/tmp/tmpiv_prys6/rcl8d6xb.json', 'init=/tmp/tmpiv_prys6/5ejzaf4r.json', 'output', 'file=/tmp/tmpiv_prys6/prophet_modelq5fxfn_y/prophet_model-20231031075301.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:53:01 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:53:01 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpiv

최적의 하이퍼파라미터: {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 1.0, 'holidays_prior_scale': 0.01}
최적의 RMSE: 3.529711374289229


In [4]:
from sklearn.model_selection import train_test_split
from prophet import Prophet

data['y'] = np.log(data['y'])

# 훈련 데이터셋과 테스트 데이터셋으로 나누기
train_df, test_df = train_test_split(data, test_size=0.2, shuffle=False)

# 모델 생성 및 학습
model = Prophet(changepoint_prior_scale=0.001, seasonality_prior_scale=1.0, holidays_prior_scale=0.1)
model.add_regressor('AvgT')
model.add_regressor('MaxWdSd')
model.add_regressor('AvgHu')
model.add_regressor('Sum_DL')
model.add_regressor('50CM_SM')
model.add_regressor('Avg_GT')
model.add_regressor('Avg_IM')
model.add_regressor('Rain')
model.fit(train_df)


# 예측할 기간 설정 및 예측 실행
future = model.make_future_dataframe(periods=len(test_df), freq='D', include_history=True)
for col in columns_to_convert:
    if col != 'y':
        future[col] = data[col]

forecast = model.predict(future)

# # 결과 확인
# model.plot(forecast)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpbmjc47tn/d7tkqwsa.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpbmjc47tn/fn29cqlj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=2679', 'data', 'file=/tmp/tmpbmjc47tn/d7tkqwsa.json', 'init=/tmp/tmpbmjc47tn/fn29cqlj.json', 'output', 'file=/tmp/tmpbmjc47tn/prophet_model39n72akw/prophet_model-20231031081341.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:13:41 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:13:41 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

# 실제값과 예측값을 원래 스케일로 변환
y_true = np.exp(test_df['y'])
y_pred = np.exp(forecast['yhat'][train_df.shape[0]:])

# MAE 계산
mae = mean_absolute_error(y_true, y_pred)
print('MAE: ', mae)

# MSE 계산
mse = mean_squared_error(y_true, y_pred)
print('MSE: ', mse)

# RMSE 계산
rmse = sqrt(mse)
print('RMSE: ', rmse)

# MAPE 계산
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print('MAPE: ', mape)


MAE:  2.772598578872633
MSE:  13.708347238998238
RMSE:  3.702478526473616
MAPE:  17.239754456510052


비지도학습 모델 이상치 탐지법

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import ParameterGrid
import numpy as np

# 데이터셋을 준비합니다.
X = pd.read_csv("/content/drive/MyDrive/SWIG/보성군_AWS_및_기상_변수선정_및_전처리본_Eng.csv")

columns_to_convert = X.columns[X.columns != 'ds']
X[columns_to_convert] = X[columns_to_convert].astype(float)

X = X.set_index('ds')

# 탐색할 'contamination' 파라미터 후보들을 정의합니다.
param_grid = {
    'contamination': [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019, 0.02]  # 예시 값입니다. 실제로는 문제에 맞게 설정해야 합니다.
}

# 파라미터 그리드를 생성합니다.
grid = ParameterGrid(param_grid)

best_score = np.inf
best_params = None

# 각 파라미터 조합에 대해 모델을 학습하고 성능을 평가합니다.
for params in grid:
    model = IsolationForest(contamination=params['contamination'])
    model.fit(X)
    score = -model.decision_function(X).mean()  # 더 낮은 점수가 더 좋은 성능을 의미합니다.
    if score < best_score:
        best_score = score
        best_params = params

# 최적의 'contamination' 파라미터 값과 그 때의 성능을 출력합니다.
print("최적의 'contamination' 파라미터 값:", best_params)
print("최적의 성능:", best_score)


In [ ]:
from sklearn.ensemble import IsolationForest

# IsolationForest 모델 생성
clf = IsolationForest(contamination=0.002)  # contamination은 이상치의 비율을 나타냅니다.

# 'y' 컬럼에 대해 IsolationForest 모델 학습 및 예측
clf.fit(data[['y']])
pred = clf.predict(data[['y']])

# 이상치를 -1로, 정상 데이터를 1로 표시합니다.
data['anomaly'] = pred

# 이상치 확인
outliers = data[data['anomaly'] == -1]
print(outliers)
